[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dakimura/jquants-sample/blob/main/data_retrieve.ipynb)

# J-Quants APIを用いた分析用データ取得

本ノートブックでは、データ分析に先立って
分析で必要となる株価・財務情報データをJ-Quants APIを使用して取得し、
保存するところまでを行います。
Google Colab上で動作確認を行っています。


## リフレッシュトークンの取得
J-Quants APIを利用するためには [J-Quants API の Web サイト](https://application.jpx-jquants.com/) から取得できるリフレッシュトークンが必要になります。
会員登録とリフレッシュトークンの取得を完了させてから次に進みます。

必要なライブラリのinstall/importと設定を行います。

In [2]:
!pip install numpy pandas requests jquants-api-client==0.1.2

import getpass
from requests import HTTPError
from datetime import datetime

import jquantsapi
import pandas as pd
import numpy as np
from dateutil import tz

# --- コンフィグ ---
# Googleドライブをマウントするディレクトリ
GOOGLE_DRIVE_MOUNT_DIR_PATH = "/content/drive/"

# データを保存するGoogleドライブ上のディレクトリ
STORAGE_DIR_PATH = "/content/drive/MyDrive/drive_ws/marketdata"

# 各種CSVデータを保存するファイルパス
stock_fins_csvfile_path = STORAGE_DIR_PATH + "/stock_fin.csv.gz"
stock_price_csvfile_path = STORAGE_DIR_PATH + "/stock_price.csv.gz"

# J-Quants API から取得するデータの期間
start_dt: datetime = datetime(2017, 1, 1)
# デバッグ中は短い期間を使う
#start_dt: datetime = datetime(2022, 3, 1)

end_dt: datetime = datetime(2022, 8, 31)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


入力ボックスに https://application.jpx-jquants.com/ から取得したJ-Quants APIのリフレッシュトークンを入力してください。

In [4]:
refresh_token = getpass.getpass(prompt="J-Quants Refresh Token:")
# リフレッシュトークンを使用できるか検証します。
test_cli = jquantsapi.Client(refresh_token=refresh_token)
try:
    id_token = test_cli.get_id_token()
    if len(id_token) > 0:
        print("refresh_tokenは正常です。次の手順に進んでください。")
except HTTPError:
    print("refresh_tokenを使用できません。再度値を確認してください。")

J-Quants Refresh Token:··········
refresh_tokenは正常です。次の手順に進んでください。


## データのダウンロード

データ取得期間(start_dt <= 時刻 <= end_dt) の
全銘柄の価格と財務情報データを取得します。
このステップの実行には環境によって数十分ほど時間がかかります。

In [6]:
cli = jquantsapi.Client(refresh_token=refresh_token)
stock_price_load: pd.DataFrame = cli.get_price_range(start_dt=start_dt, end_dt=end_dt)
stock_fin_load: pd.DataFrame = cli.get_statements_range(start_dt=start_dt, end_dt=end_dt)

100 / 184


## データ処理

過去にJ-Quants株式分析チュートリアルやデータ分析コンペティションで用いられたデータとの互換性をなるべく取るため、データの処理を行います。

In [11]:
# 財務情報のいくつかがobject型になっているので数値型に変換
numeric_cols_fin = ['AverageNumberOfShares', 'BookValuePerShare', 'EarningsPerShare','Equity', 'EquityToAssetRatio',
                'ForecastDividendPerShare1stQuarter', 'ForecastDividendPerShare2ndQuarter', 'ForecastDividendPerShare3rdQuarter',
                'ForecastDividendPerShareAnnual', 'ForecastDividendPerShareFiscalYearEnd', 'ForecastEarningsPerShare', 'ForecastNetSales', 'ForecastOperatingProfit',
                'ForecastOrdinaryProfit', 'ForecastProfit', 'NetSales', 'NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock',
                'OperatingProfit', 'OrdinaryProfit', 'Profit', 'ResultDividendPerShare1stQuarter','ResultDividendPerShare2ndQuarter','ResultDividendPerShare3rdQuarter',
                'ResultDividendPerShareAnnual','ResultDividendPerShareFiscalYearEnd','TotalAssets']
stock_fin_load[numeric_cols_fin] = stock_fin_load[numeric_cols_fin].apply(pd.to_numeric, errors='coerce', axis=1)

# object型をdatetime64[ns]型に変換
stock_price_load["Date"] = pd.to_datetime(stock_price_load["Date"])
stock_fin_load["DisclosedDate"] = pd.to_datetime(stock_fin_load["DisclosedDate"]) #開示
stock_fin_load["CurrentFiscalYearEndDate"] = pd.to_datetime(stock_fin_load["CurrentFiscalYearEndDate"])  # 当事業年度終了日
stock_fin_load["CurrentFiscalYearStartDate"] = pd.to_datetime(stock_fin_load["CurrentFiscalYearStartDate"])  # 当事業年度開始日
stock_fin_load["CurrentPeriodEndDate"] = pd.to_datetime(stock_fin_load["CurrentPeriodEndDate"]) # 当会計期間終了日

In [12]:
# stock_price: データの互換性のための各種列名変換など
stock_price: pd.DataFrame = pd.DataFrame()
stock_price["Local Code"] = stock_price_load["Code"]
#stock_price["Date"] = stock_price_load["Date"]
stock_price["base_date"] = stock_price_load["Date"]
stock_price['EndOfDayQuote Date'] = stock_price_load["Date"]
stock_price["EndOfDayQuote Open"] = stock_price_load["AdjustmentOpen"].replace({0.0: np.nan})
stock_price["EndOfDayQuote High"] = stock_price_load["AdjustmentHigh"].replace({0.0: np.nan})
stock_price["EndOfDayQuote Low"] = stock_price_load["AdjustmentLow"].replace({0.0: np.nan})
stock_price["EndOfDayQuote Close"] = stock_price_load["AdjustmentClose"].replace({0.0: np.nan})
stock_price["EndOfDayQuote ExchangeOfficialClose"] = stock_price_load["AdjustmentClose"].replace({0.0: np.nan})
stock_price["EndOfDayQuote Volume"] = stock_price_load["AdjustmentVolume"]

# stock_price["EndOfDayQuote Open"][stock_price["EndOfDayQuote Close"] == 0] = \
#     stock_price["EndOfDayQuote ExchangeOfficialClose"]
# stock_price["EndOfDayQuote High"][stock_price["EndOfDayQuote Close"] == 0] = \
#     stock_price["EndOfDayQuote ExchangeOfficialClose"]
# stock_price["EndOfDayQuote Low"][stock_price["EndOfDayQuote Close"] == 0] = \
#     stock_price["EndOfDayQuote ExchangeOfficialClose"]
# stock_price["EndOfDayQuote Close"][stock_price["EndOfDayQuote Close"] == 0] = \
#     stock_price["EndOfDayQuote ExchangeOfficialClose"]

# 前日終値の列を終値列から作成
stock_price["EndOfDayQuote PreviousClose"] = stock_price.groupby(["Local Code"])["EndOfDayQuote Close"].shift(1)

In [13]:
# stock_financial: データの互換性のための各種列名変換など
stock_fin: pd.DataFrame = pd.DataFrame()
stock_fin["Local Code"] = stock_fin_load["LocalCode"]
stock_fin["Result_FinancialStatement FiscalPeriodEnd"] = stock_fin_load["CurrentFiscalYearEndDate"]
stock_fin["Result_FinancialStatement TotalAssets"] = stock_fin_load["TotalAssets"] # 総資産
stock_fin["Result_FinancialStatement NetAssets"] = stock_fin_load["Equity"] # 純資産
stock_fin["Result_FinancialStatement NetSales"] = stock_fin_load["NetSales"] # 純売上高
stock_fin["Result_FinancialStatement OperatingIncome"] = stock_fin_load["OperatingProfit"] # 営業利益
stock_fin["Result_FinancialStatement OrdinaryIncome"] = stock_fin_load["OrdinaryProfit"]  # 経常利益
stock_fin["Result_FinancialStatement NetIncome"] = stock_fin_load["Profit"]  # 当期純利益
stock_fin["Result_FinancialStatement ReportType"] = stock_fin_load["TypeOfCurrentPeriod"] # {"1Q", "2Q", "3Q", "FY"}
stock_fin["base_date"] = stock_fin_load["DisclosedDate"] # 開示日
stock_fin["TypeOfDocument"] = stock_fin_load["TypeOfDocument"] # 書類種別
stock_fin["RetrospectiveRestatement"] = stock_fin_load["RetrospectiveRestatement"] #修正再表示フラグ
stock_fin["Forecast_FinancialStatement FiscalPeriodEnd"] = stock_fin_load["CurrentFiscalYearEndDate"]
stock_fin["Forecast_FinancialStatement ReportType"] = stock_fin_load["TypeOfCurrentPeriod"]
stock_fin["Forecast_FinancialStatement NetSales"] = stock_fin_load["ForecastNetSales"]
stock_fin["Forecast_FinancialStatement OperatingIncome"] = stock_fin_load["ForecastOperatingProfit"]
stock_fin["Forecast_FinancialStatement NetIncome"] = stock_fin_load["ForecastProfit"]
stock_fin["Forecast_FinancialStatement OrdinaryIncome"] = stock_fin_load["ForecastOrdinaryProfit"]

stock_fin

,Local Code,Result_FinancialStatement FiscalPeriodEnd,Result_FinancialStatement TotalAssets,Result_FinancialStatement NetAssets,Result_FinancialStatement NetSales,Result_FinancialStatement OperatingIncome,Result_FinancialStatement OrdinaryIncome,Result_FinancialStatement NetIncome,Result_FinancialStatement ReportType,base_date,TypeOfDocument,RetrospectiveRestatement,Forecast_FinancialStatement FiscalPeriodEnd,Forecast_FinancialStatement ReportType,Forecast_FinancialStatement NetSales,Forecast_FinancialStatement OperatingIncome,Forecast_FinancialStatement NetIncome,Forecast_FinancialStatement OrdinaryIncome
5,72310,2022-03-31,NaN,NaN,NaN,NaN,NaN,NaN,FY,2022-03-01,ForecastRevision,,2022-03-31,FY,1.690000e+11,-2.700000e+09,5.000000e+08,-2.000000e+08
0,70670,2022-03-31,NaN,NaN,NaN,NaN,NaN,NaN,FY,2022-03-01,ForecastRevision,,2022-03-31,FY,NaN,NaN,NaN,NaN
1,85580,2022-03-31,NaN,NaN,NaN,NaN,NaN,NaN,FY,2022-03-01,ForecastRevision,,2022-03-31,FY,NaN,NaN,NaN,NaN
3,39940,2022-11-30,NaN,NaN,NaN,NaN,NaN,NaN,1Q,2022-03-01,ForecastRevision,,2022-11-30,1Q,NaN,NaN,NaN,NaN
4,25930,2022-04-30,3.250440e+11,1.587270e+11,3.036260e+11,1.411100e+10,1.465600e+10,1.033800e+10,3Q,2022-03-01,3QFinancialStatements_Consolidated_JP,false,2022-04-30,3Q,4.031000e+11,2.000000e+10,1.280000e+10,1.980000e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,80680,2023-01-31,NaN,NaN,NaN,NaN,NaN,NaN,FY,2022-08-31,ForecastRevision,,2023-01-31,FY,1.130000e+11,3.600000e+09,2.350000e+09,3.400000e+09
0,46530,2022-03-31,2.239600e+10,1.255300e+10,2.878300e+10,1.228000e+09,1.244000e+09,4.360000e+08,FY,2022-08-31,NumericalCorrection,false,2022-03-31,FY,3.272900e+10,1.653000e+09,1.005000e+09,1.514000e+09
4,43710,2022-12-31,4.400282e+09,2.506768e+09,7.169660e+08,6.906530e+08,7.147880e+08,NaN,2Q,2022-08-31,NumericalCorrection,,2022-12-31,2Q,NaN,NaN,NaN,NaN
5,46530,2023-03-31,NaN,NaN,NaN,NaN,NaN,NaN,FY,2022-08-31,ForecastRevision,,2023-03-31,FY,NaN,NaN,NaN,NaN


## GoogleDriveへのCSV保存

取得したデータをcsv.gz形式で保存します。
stock_priceが巨大なファイル（非圧縮, 2017-01-01〜2022-07-31の期間だと540MB程度)なので、
保存には3分ほどかかることがあります。

In [14]:
stock_price.to_csv(stock_price_csvfile_path, compression="gzip")
stock_fin.to_csv(stock_fins_csvfile_path, compression="gzip")

このnotebookはデータ保存までで終了です。predictor.ipynbに進みましょう。